In [23]:
from espnet2.bin.tts_inference import Text2Speech
from espnet2.utils.types import str_or_none
from IPython.display import Audio
from urllib.parse import quote
from pydub import AudioSegment
from bs4 import BeautifulSoup
from tqdm import tqdm
import io
import time
import torch
import soundfile as sf
import requests
import warnings
import json
warnings.simplefilter("ignore", UserWarning)

In [8]:
def get_tukuyomi(text:str, is_save=False):
    """
    ! python main.py
    で実行したAPIサーバーにリクエストを投げる
    """
    res = requests.get(f"http://localhost:5001/tts?text={quote(text)}")
    # 既に取得した res.content を使用
    if res.status_code == 200:
        audio = Audio(res.content, autoplay=True)
        display(audio)
        if is_save:
            with open(f"./data/{text.replace('/','_')}.wav", "wb") as f:
                f.write(res.content)
                print("ファイルを保存しました。")
    else:
        print(f"エラーが発生しました: ステータスコード {res.status_code}")

In [ ]:
get_tukuyomi("最近話題のTTS試してみた")

In [ ]:
quotes = []
for i in tqdm(range(1,18)):
    res = requests.get(f"https://iyashitour.com/meigen/theme/life/{i}")
    soup = BeautifulSoup(res.text, 'html.parser')
    quote_elms = soup.find_all('p', {'class':'quote'})
    quotes += [q.get_text(" ", strip=True) for q in quote_elms]

with open('./data/quotes.txt', 'w') as f:
    for line in quotes:
        f.write(f"{line}\n")

In [21]:
with open("./data/quotes.txt","r") as f:
    # 最初の音声ファイルを基点として、他の音声データを結合します
    quotes = [q for q in f]
    combined = AudioSegment.empty()
    for text in tqdm(quotes):
        res = requests.get(f"http://localhost:5001/tts?text={quote(text)}")
        if res.status_code == 200:
            content = res.content
            audio_segment = AudioSegment.from_file(io.BytesIO(content), format="wav")
            # 無声音
            a = AudioSegment.silent(duration=1000)
            combined += audio_segment+a
    # 結合した音声ファイルを新しいファイルとして出力
    combined.export("./data/quotes.wav", format="wav")

100%|██████████| 270/270 [07:56<00:00,  1.77s/it]


<_io.BufferedRandom name='./data/quotes.wav'>

In [ ]:
Audio("./data/quotes.wav")